# client api

- https://duckdb.org/docs/stable/clients/python/dbapi

In [10]:
import duckdb

# duckdb.execute("CREATE TABLE tbl AS SELECT 42 a")
con = duckdb.connect(":memory:")

con.execute("CREATE TABLE tbl AS SELECT 42 a")

con.sql("SELECT * FROM tbl")


# or
# duckdb.default_connection.sql("SELECT * FROM tbl")

┌───────┐
│   a   │
│ int32 │
├───────┤
│    42 │
└───────┘

In [ ]:
import duckdb

# to start an in-memory database
con = duckdb.connect(database=":memory:")

# to use a database file (not shared between processes)
con = duckdb.connect(database="my-db.duckdb", read_only=False)

# to use a database file (shared between processes)
con = duckdb.connect(database="my-db.duckdb", read_only=True)

# to explicitly get the default connection
con = duckdb.connect(database=":default:")

In [ ]:
# create a table
con.execute("CREATE TABLE items (item VARCHAR, value DECIMAL(10, 2), count INTEGER)")
# insert two items into the table
con.execute("INSERT INTO items VALUES ('jeans', 20.0, 1), ('hammer', 42.2, 2)")

# retrieve the items again
con.execute("SELECT * FROM items")
print(con.fetchall())
# [('jeans', Decimal('20.00'), 1), ('hammer', Decimal('42.20'), 2)]

# retrieve the items one at a time
con.execute("SELECT * FROM items")

print(con.fetchone())
# ('jeans', Decimal('20.00'), 1)

print(con.fetchone())
# ('hammer', Decimal('42.20'), 2)


print(
    con.fetchone()
)  # This closes the transaction. Any subsequent calls to .fetchone will return None
# None

[('jeans', Decimal('20.00'), 1), ('hammer', Decimal('42.20'), 2)]
('jeans', Decimal('20.00'), 1)
('hammer', Decimal('42.20'), 2)
None


In [52]:
import duckdb

# to start an in-memory database
con = duckdb.connect(database=":memory:")

# create a table
con.execute("CREATE TABLE items (item VARCHAR, value DECIMAL(10, 2), count INTEGER)")


# insert two items into the table
con.execute("INSERT INTO items VALUES ('jeans', 20.0, 1), ('hammer', 42.2, 2)")

con.execute("INSERT INTO items VALUES (?, ?, ?)", ["laptop", 2000, 1])

con.execute("SELECT * FROM items").df()

,item,value,count
0,jeans,20.0,1
1,hammer,42.2,2
2,laptop,2000.0,1


In [46]:
con.execute("SELECT * FROM items").df()

,item,value,count
0,jeans,20.0,1
1,hammer,42.2,2
2,laptop,2000.0,1


In [47]:
con.execute("SELECT * FROM items").pl()

# ret.fetchnumpy()

item,value,count
str,"decimal[10,2]",i32
"""jeans""",20.00,1
"""hammer""",42.20,2
"""laptop""",2000.00,1


In [48]:
con.execute("SELECT * FROM items").arrow()

pyarrow.Table
item: string
value: decimal128(10, 2)
count: int32
----
item: [["jeans","hammer","laptop"]]
value: [[20.00,42.20,2000.00]]
count: [[1,2,1]]

In [49]:
con.execute("SELECT * FROM items").fetchall()

[('jeans', Decimal('20.00'), 1),
 ('hammer', Decimal('42.20'), 2),
 ('laptop', Decimal('2000.00'), 1)]

In [53]:
con.executemany(
    "INSERT INTO items VALUES (?, ?, ?)", [["chainsaw", 500, 10], ["iphone", 300, 2]]
)


con.execute("SELECT * FROM items").df()

,item,value,count
0,jeans,20.0,1
1,hammer,42.2,2
2,laptop,2000.0,1
3,chainsaw,500.0,10
4,iphone,300.0,2


In [55]:
con.execute("SELECT item FROM items WHERE value > ?", [400])

con.df()
# print(con.fetchall())

,item
0,laptop
1,chainsaw


In [58]:
con.execute("SELECT $1, $1, $2", ["duck", "goose"])

con.df()
# print(con.df())
# print(con.fetchall())

,$1,$1_1,$2
0,duck,duck,goose


In [60]:
import duckdb

res = duckdb.execute(
    """
    SELECT
        $my_param,
        $other_param,
        $also_param
    """,
    {"my_param": 5, "other_param": "DuckDB", "also_param": [42]},
)

res.df()

,$my_param,$other_param,$also_param
0,5,DuckDB,[42]
